# Build your own model
## Pre-Requirements (30 - 40 Minutes)

1. Create a cloud 9 instance and extend volume to 50G
2. Create a S3 bucket, such as smworkshop-cszhz, 
 then create a subfolder names <b>ctpn</b> under smworkshop-cszhz,
 then create a subfolder names <b>results</b> under ctpn
 last upload some test images into <b>ctpn</b> folder
3. Download the code from github (http://git.awsrun.com/Damon/gcr_sm_workshop/tree/master/image_video/zz/work) and upload to cloud 9 instance you just created
4. Update <b>region, s3, accesskey and secretkey</b> information from file <b>work\ctpn\ctpn\awss3.py </b>
4. Download model file from S3 ( https://smworkshop-cszhz.s3.amazonaws.com/ctpn/checkpoints.zip ) and upload to cloud 9, then extract it into work folder
5. Enter work folder, and run
  ./build.sh ctpn
6. When you complete above process, then continue 

## Deploy your model (10 - 20 Minutes)

In [1]:
# S3 prefix
prefix = 'smworkshop-cszhz'

# Define IAM role
import boto3
import re

import os
import numpy as np
import pandas as pd
from sagemaker import get_execution_role

role = get_execution_role()
print(role)

arn:aws:iam::549828897912:role/service-role/AmazonSageMaker-ExecutionRole-20190814T164000


Create SageMaker session.

In [2]:
import sagemaker as sage
from time import gmtime, strftime

sess = sage.Session()

Get image url from ecr. 
In this excercise, the image name is ctpn

In [3]:
account = sess.boto_session.client('sts').get_caller_identity()['Account']
region = sess.boto_session.region_name
image = '{}.dkr.ecr.{}.amazonaws.com/ctpn:latest'.format(account, region)
print(image)

549828897912.dkr.ecr.us-east-1.amazonaws.com/ctpn:latest


Call SageMaker API to create model, you also is able to create it from aws console

In [ ]:
model = sess.create_model(
        name="CTPN", role=role, container_defs=image)
print(model)


Call SageMaker API to create endpoint configuration, you also is able to create it from aws console

In [ ]:
endpoint_cfg=sess.create_endpoint_config(
        name="CTPNCFG",
        model_name="CTPN",
        initial_instance_count=1,
        instance_type="ml.c4.xlarge"
    )
print(endpoint_cfg)

Call SageMaker API to create endpoint, you also is able to create it from aws console.
This step is an asynchronous call and always take ten minutes, before next step you may want to drink a cup of coffee.

In [ ]:
endpoint=sess.create_endpoint(
        endpoint_name="CTPNEndpoint", config_name="CTPNCFG")
print(endpoint)

## Test your model (3 Minutes)

Below code will test the deployed ctpn model.

In [8]:
#imgName="1.png"
imgName="3.jpg"

In [9]:
#display original image
from IPython.display import Image
Image(url= "https://"+prefix+".s3.amazonaws.com/ctpn/"+imgName)

In [10]:
runtime = boto3.Session().client('runtime.sagemaker')
payload="fn="+imgName
endpoint="CTPNEndpoint"
response = runtime.invoke_endpoint(EndpointName=endpoint,ContentType="application/x-www-form-urlencoded",Body=payload)
result=response['Body'].read()
print(result)

Detection took 7.162s for 17 object proposals


In [11]:
from IPython.display import Image
Image(url= "https://"+prefix+".s3.amazonaws.com/ctpn/results/"+imgName)